In [1]:
classpath.add("com.chuusai" %% "shapeless" % "2.3.0")

Adding 2 artifact(s)


In [2]:
import shapeless._
import shapeless.poly._

import shapeless._
import shapeless.poly._

# HLists

In [26]:
val myFirstHList = 1 :: "string" :: List(3.0f, 3.14f) :: HNil

myFirstHList: Int :: String :: List[Float] :: HNil = 1 :: string :: List(3.0, 3.14) :: HNil

In [4]:
object size extends Poly1 {
    implicit def default[T] = at[T](_ => 1)
    implicit def caseString = at[String](_.length)
    implicit def caseList[T] = at[List[T]](_.length)
}

defined object size

In [5]:
val sizes = myFirstHList.map(size)

sizes: Int :: Int :: Int :: HNil = 1 :: 6 :: 2 :: HNil

In [6]:
sizes.toList

res5: List[Int] = List(1, 6, 2)

In [7]:
myFirstHList zip sizes

res6: (Int, Int) :: (String, Int) :: (List[Float], Int) :: HNil = (2,1) :: (string,6) :: (List(3.0, 3.14),2) :: HNil

## Case classes

In [8]:
case class MyLittleCaseClass(x: Int, y: String, z: Float)

defined class MyLittleCaseClass

In [9]:
val myFirstValue = MyLittleCaseClass(1, "string", 3.0f)

myFirstValue: MyLittleCaseClass = MyLittleCaseClass(1, "string", 3.0F)

In [10]:
import shapeless.syntax.std.product._
myFirstValue.toHList

import shapeless.syntax.std.product._
res9_1: Int :: String :: Float :: HNil = 1 :: string :: 3.0 :: HNil

## Generic programming

In [11]:
val gen = Generic[MyLittleCaseClass]

gen: Generic[MyLittleCaseClass]{type Repr = shapeless.::[Int,shapeless.::[String,shapeless.::[Float,shapeless.HNil]]]} = cmd10$$user$$anonfun$1$anon$macro$179$1@7e51348f

In [12]:
val asHList = gen.to(myFirstValue)

asHList: gen.Repr = 1 :: string :: 3.0 :: HNil

In [13]:
val andBack = gen.from(asHList)

andBack: MyLittleCaseClass = MyLittleCaseClass(1, "string", 3.0F)

# Optics

In [14]:
val xLens = lens[MyLittleCaseClass].x

xLens: Lens[MyLittleCaseClass, Int] = shapeless.Lens$$anon$7@6f39f382

In [15]:
xLens.get(myFirstValue)

res14: Int = 1

In [16]:
xLens.modify(myFirstValue)(_ + 1)

res15: MyLittleCaseClass = MyLittleCaseClass(2, "string", 3.0F)

In [17]:
val headPrism = prism[List[Int]].head

headPrism: Prism[List[Int], Int] = shapeless.Prism$$anon$10@60d3eb72

In [18]:
headPrism.get(List(1, 2))

res17: Option[Int] = Some(1)

In [19]:
headPrism.get(Nil)

res18: Option[Int] = None

In [20]:
headPrism.modify(List(1, 2))(_ + 1)

res19: List[Int] = List(2, 2)

In [21]:
headPrism.modify(Nil)(_ + 1)

res20: List[Int] = List()

# Type Class Derivation

In [22]:
trait Monoid[T] {
    def zero: T
    def append(a: T, b: T): T
}


object Monoid extends ProductTypeClassCompanion[Monoid] {
    implicit def intMonoid: Monoid[Int] = new Monoid[Int] {
        def zero = 0
        def append(a: Int, b: Int) = a+b
    }

    implicit def stringMonoid: Monoid[String] = new Monoid[String] {
        def zero = ""
        def append(a: String, b: String) = a+b
    }

    object typeClass extends ProductTypeClass[Monoid] {
        def emptyProduct = new Monoid[HNil] {
            def zero = HNil
            def append(a: HNil, b: HNil) = HNil
        }

        def product[F, T <: HList](mh: Monoid[F], mt: Monoid[T]) = new Monoid[F :: T] {
            def zero = mh.zero :: mt.zero
            def append(a: F :: T, b: F :: T) = mh.append(a.head, b.head) :: mt.append(a.tail, b.tail)
        }

        def project[F, G](instance: => Monoid[G], to: F => G, from: G => F) = new Monoid[F] {
            def zero = from(instance.zero)
            def append(a: F, b: F) = from(instance.append(to(a), to(b)))
        }
    }
}

defined trait Monoid
defined object Monoid

In [23]:
case class Vector2(x: Int, y: Int)
object Vector2 {
    def instance: Monoid[Vector2] = implicitly
}

defined class Vector2
defined object Vector2

In [24]:
Vector2.instance.zero

res23: Vector2 = Vector2(0, 0)

In [25]:
Vector2.instance.append(Vector2(1, 1), Vector2(1, 2))

res24: Vector2 = Vector2(2, 3)